# Clustering

### Classifying heartbeat anomalies from stethoscope audio

Use BregmanToolkit to extract a feature vector out of our sound.
Download BregmanToolkit: https://github.com/arturogro/BregmanToolkit.git

In [73]:
%matplotlib inline
import tensorflow as tf
import numpy as np
from bregman.suite import *

Set the hyper-paramaters for clustering

In [74]:
k = 4
max_iterations = 100

Select the locations for the audio files

In [75]:
filenames = tf.train.match_filenames_once('./dataset/set-a/*.wav')
count_num_files = tf.size(filenames)
filename_queue = tf.train.string_input_producer(filenames)
reader = tf.WholeFileReader()
filename, file_contents = reader.read(filename_queue)

Create an op to identify the pitch with the biggest contribution

In [76]:
chromo = tf.placeholder(tf.float32)
max_freqs = tf.argmax(chromo, 0)

Create a helper function to get the next audio file's Chromogram:

In [77]:
def get_next_chromogram(sess):
    audio_file = sess.run(filename)
    print('Loading {}'.format(audio_file))
    F = Chromagram(audio_file, nfft=16384, wfft=8192, nhop=2205)
    return F.X, audio_file

Create a helper function to extract a feature vector from the Chromogram data:

In [78]:
def extract_feature_vector(sess, chromo_data):
    num_features, num_samples = np.shape(chromo_data)
    freq_vals = sess.run(max_freqs, feed_dict={chromo: chromo_data})
    hist, bins = np.histogram(freq_vals, bins=range(num_features + 1))
    return hist.astype(float)  / num_samples

Extract a dataset of feature vectors by calling our helper functions above:

In [79]:
def get_dataset(sess):
    num_files = sess.run(count_num_files)
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    xs = list()
    names = list()
    
    for _ in range(num_files):
        plt.figure()
        chromo_data, filename = get_next_chromogram(sess)
        
        plt.subplot(1, 2, 1)
        plt.imshow(chromo_data, cmap='Greys', interpolation='nearest')
        plt.title('Visualization of Sound Spectrum')
        
        plt.subplot(1, 2, 2)
        freq_vals = sess.run(max_freqs, feed_dict={chromo: chromo_data})
        plt.hist(freq_vals)
        plt.title('Histogram of Notes')
        plt.xlabel('Musical Note')
        plt.ylabel('Count')
        plt.savefig('{}.png'.format(filename))
        plt.show()
        
        names.append(filename)
        x = extract_feature_vector(sess, chromo_data)
        xs.append(x)
        
    xs = np.asmatrix(xs)
    coord.request_stop()
    coord.join(threads)
    return xs, names

k-means clustering requires 2 main phases: clustering and assignment. We'll also throw in an initialization helper function for good measure. Here are the three components:

In [80]:
def initial_cluster_centroids(X, k):
    return X[0:k, :]

def assign_cluster(X, centroids):
    expanded_vectors = tf.expand_dims(X, 0)
    expanded_centroids = tf.expand_dims(centroids, 1)
    distances = tf.reduce_sum(tf.square(tf.subtract(expanded_vectors, expanded_centroids)),2)
    mins = tf.argmin(distances, 0)
    return mins

def recompute_centroids(X, Y):
    sums = tf.unsorted_segment_sum(X, Y, k)
    counts = tf.unsorted_segment_sum(tf.ones_like(X), Y, k)
    return sums / counts

Open a session, obtain a dataset, and cluster the data:

In [81]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    X, names = get_dataset(sess)
    centroids = initial_cluster_centroids(X, k)
    i, converged = 0, False
    while not converged and i < max_iterations:
        i += 1
        Y = assign_cluster(X, centroids)
        centroids = sess.run(recompute_centroids(X, Y))
    for y, name in zip(sess.run(Y), names):
        print(y, name)

NotFoundError: ./audio_dataset/set_a
	 [[Node: matching_filenames/MatchingFiles = MatchingFiles[_device="/job:localhost/replica:0/task:0/cpu:0"](matching_filenames/MatchingFiles/pattern)]]

Caused by op u'matching_filenames/MatchingFiles', defined at:
  File "/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/aj.gupa/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/aj.gupa/tensorflow/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/aj.gupa/tensorflow/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/Users/aj.gupa